In [2]:
from ortools.sat.python import cp_model

# Asignación de tareas con periodos de mantenimientos por uso acumulativo

El objetivo de este modelo es asignar un calendario de trabajos a un conjunto de maquina con la premisa de que cada máquina debe pasarpor un mantenimiento preventivo cada 80 horas de uso acumulado.

En esta oportunidad se necesitan al menos 4 maquinas para cumplir con la cuota productiva, se disponen de 4 hangares de mantenimiento y se busca generar un calendario de trabajo para 10 máquinas cubriendo 24 periodos de trabajo (cada periodo de trabajo puede ser de 8 horas por lo que los 24 periodos hace referencia a 8 días de programación)

Del mismo modo se puede utilizar el algebra para modelar un problema asi, esto dara una noción acertada de lo que se busca crear pero con la ayuda de las funciones de ortools el diseño será intuitivo.

- X<sub>i, t</sub>  Es 1 si la máquina i es asignada a trabajar en el periodo t 0 \~
- M<sub>i, t</sub> Es 1 si la máquina i es asignada a un mantenimiento en el periodo t 0\~
- A<sub>i,T</sub> Representa la suma acumulativa de la cantidad de periodos trabajados ($\sum_{i}^{T}X_{i,T}$)

Nuevamente para asignar las horas de trabajo a las máquinas necesitamos saber que tipo de función vamos a optimizar. En esta oportunidad podemos considerar que mantener una mínima fila de máquinas esperando el mantenimiento nos indicaría que estamos haciendo un uso mesurado de los equipos. 

¿Cual es el argumento de esta premisa?
* Al tener la mayor cantidad de máquinas operativas le doy resiliencia a los paros a la linea de producción. Dado que siempre tendre el máximo número de maquinas disponibles para cubrir los turnos

\begin{aligned}
    &Z= \min(\sum_{t=0}^{20}\sum_{i}^{M} (M_{i,t}) )_{i \in [maquina_{1}, \ldots, maquina_{n}] }\\
    & \text{sujeto a} \\
    &  \sum_{i}^{M}X_{i, T}=\text{necesidad operativa} \; \forall \: i \in Maquinas\\
    &  \sum_{i}^{M} M{i, T}=\text{hangares disponibles} \; \forall \: i \in Maquinas\\
    & \sum_{i=1}^{M} X_{i,T} + M_{i, T}\leq 1 \;  \forall \: i \in Maquinas \\
    &  \sum_{t}^{T}A_{I,t} = \sum_{t}^{T} A_{I,t-1} + X_{I, t}\\
\end{aligned}

In [3]:
# Parámetros del problema
num_maquinas = 10
necesidad_operativa = 4
tiempo_trabajo = 8
num_periodos = 20  
umbrales_mantenimiento = [80]  # 80 horas
hangares_disponibles = 4 

In [4]:
model = cp_model.CpModel()

# Variables
x = {}
for i in range(num_maquinas):
    for t in range(num_periodos):
        x[i, t] = model.NewBoolVar(f'x[{i},{t}]')  # Si la máquina i trabaja en el periodo t

mantenimiento = {}
for i in range(num_maquinas):
    for t in range(num_periodos):
        mantenimiento[i, t] = model.NewBoolVar(f'mantenimiento[{i},{t}]')  # Si la máquina i está en mantenimiento en el periodo t

acumulado = {}
acumulado_real = {}
for i in range(num_maquinas):
    for t in range(num_periodos):
        acumulado[i, t] = model.NewIntVar(0, max(umbrales_mantenimiento)//tiempo_trabajo, f'acumulado[{i},{t}]')  # Horas acumuladas de la máquina i
        acumulado_real[i, t] = model.NewIntVar(0, max(umbrales_mantenimiento)//tiempo_trabajo, f'acumulado_real[{i},{t}]')  # Valor efectivo del acumulado

# Restricciones
for t in range(num_periodos):
    model.Add(sum(x[i, t] for i in range(num_maquinas)) == necesidad_operativa)

    # No más  hangares_disponibles que máquinas en mantenimiento
    model.Add(sum(mantenimiento[i, t] for i in range(num_maquinas)) <= hangares_disponibles)

for i in range(num_maquinas):
    for t in range(num_periodos):
        # Una máquina no puede trabajar y estar en mantenimiento al mismo tiempo
        model.Add(x[i, t] + mantenimiento[i, t] <= 1)

        if t == 0:
            # Acumulado inicial es el trabajo en el primer periodo
            model.Add(acumulado[i, t] == x[i, t])
        else:
            # Acumulado sin mantenimiento
            model.Add(acumulado_real[i, t] == acumulado[i, t - 1] + x[i, t])

            # Acumulado efectivo con mantenimiento (se resetea si hay mantenimiento)
            model.Add(acumulado[i, t] == acumulado_real[i, t]).OnlyEnforceIf(mantenimiento[i, t].Not())
            model.Add(acumulado[i, t] == 0).OnlyEnforceIf(mantenimiento[i, t])

        # Si alcanza el umbral de mantenimiento, debe parar para mantenimiento
        for umbral in umbrales_mantenimiento:
            model.Add(acumulado[i, t] < umbral + mantenimiento[i, t] * (umbral + 1))

# Función objetivo: minimizar periodos de espera para mantenimiento
model.Minimize(sum(mantenimiento[i, t] for i in range(num_maquinas) for t in range(num_periodos)))


In [5]:
# Solucionador
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 120
status = solver.Solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("Solución encontrada:")
    for i in range(num_maquinas):
        periodos = []
        for t in range(num_periodos):
            if solver.Value(x[i, t]) == 1:
                periodos.append(t)

        # Agrupar periodos consecutivos y mostrar en formato legible
        horarios = []
        inicio = None
        for j in range(len(periodos)):
            if inicio is None:
                inicio = periodos[j]
            if j == len(periodos) - 1 or periodos[j + 1] != periodos[j]:
                if inicio == periodos[j]:
                    horarios.append(f"{inicio}")
                else:
                    horarios.append(f"{inicio} {periodos[j]}")
                if j < len(periodos) - 1:
                    horarios.append("...")
                inicio = None
        print(f"Maquina_{i+1}: {' '.join(horarios)}")
else:
    print("No se encontró solución factible.")

Solución encontrada:
Maquina_1: 1 ... 3 ... 5 ... 6 ... 7 ... 8 ... 12 ... 13 ... 15 ... 16
Maquina_2: 1 ... 2 ... 4 ... 5 ... 6 ... 9 ... 10 ... 11 ... 16 ... 19
Maquina_3: 2 ... 4 ... 5 ... 6 ... 10 ... 11 ... 13 ... 14 ... 15 ... 16
Maquina_4: 1 ... 3 ... 4 ... 7 ... 10 ... 11 ... 12 ... 13 ... 14 ... 18
Maquina_5: 2 ... 3 ... 7 ... 10 ... 11 ... 12 ... 13 ... 14 ... 17 ... 18
Maquina_6: 0 ... 2 ... 3 ... 4 ... 5 ... 8 ... 9 ... 12 ... 15 ... 19
Maquina_7: 0 ... 8 ... 9
Maquina_8: 0 ... 6 ... 8 ... 16 ... 17 ... 18
Maquina_9: 0 ... 1 ... 17 ... 18 ... 19
Maquina_10: 7 ... 9 ... 14 ... 15 ... 17 ... 19
